In [ ]:
'''
## Overview
Here I applies the linear regression using NHST.
I predict each ROI's activity using the angular head velocity and 
the angular head speed to detect the three types of rings: left, right,
or symmetric.

Before applying the regression, I excluded fish whose rotation on either
side is smaller than three.

After detecting the three rings, I examined their anatomical distribution 
by plotting or different statistical tests. I also plotted the activity pattern
of each ROI as a function of velocity.

Note that only cells classified as HD cells undergo this analysis.

## Author
Siyuan Mei (mei@bio.lmu.de)

## Last update
2025-9-11: add docstring for the notebook
'''
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from tqdm import tqdm

from lotr import LotrExperiment
from lotr.default_vals import TURN_BIAS
from lotr.behavior import get_bouts_props_array
from lotr.utils import convolve_with_tau
from HD_utils.HD_functions import *
from HD_utils.network import *
from HD_utils.IO import *
from HD_utils.defaults import *

from functions_12 import *

pd.options.display.max_columns = 30

fish_num = len(DATA_FOLDERS)
bin_num = 4
tau = 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Number of CW & CCW turns of each Fish

In [ ]:
bout_list = []
for fish in range(fish_num):
    exp = LotrExperiment(DATA_FOLDERS[fish])
    theta_rotation = get_bouts_props_array(exp.n_pts, exp.bouts_df, min_bias=TURN_BIAS, selection="all", value="bias")
    bout_list.append(theta_rotation)

In [6]:
for fish in range(fish_num):
    theta_rotation = bout_list[fish]
    CW_turn_num = np.sum(theta_rotation > 0)
    CCW_turn_num = np.sum(theta_rotation < 0)

CW_turn_list = [] # number of CW turns
CCW_turn_list = [] # number of CCW turns
ratio_list = [] # ratio of CCW to CW turns
for fish in range(fish_num):
    theta_rotation = bout_list[fish]
    CW_turn_num = np.sum(theta_rotation > 0)
    CCW_turn_num = np.sum(theta_rotation < 0)
    ratio = CCW_turn_num / CW_turn_num

    CW_turn_list.append(CW_turn_num)
    CCW_turn_list.append(CCW_turn_num)
    ratio_list.append(ratio)

# Transform data to datafram

Exclude fish with less than bout_thres bouts for either rotation direction 

In [ ]:
fish_exclude = []
bout_num_thres = 3 # minimum number of bouts for each direction
for fish in range(fish_num):
    exp = LotrExperiment(DATA_FOLDERS[fish])
    theta_rotation = get_bouts_props_array(exp.n_pts, exp.bouts_df, min_bias=TURN_BIAS, selection="all", value="bias")
    CW_turn_num = np.sum(theta_rotation > 0)
    CCW_turn_num = np.sum(theta_rotation < 0)
    if CW_turn_num < bout_num_thres or CCW_turn_num < bout_num_thres:
        fish_exclude.append(fish)

In [8]:
fish_exclude

[12, 20, 26, 27]

In [ ]:
# Create a dataframe to store the data for linear regression
df = pd.DataFrame(columns=["fish", "cell", "activity", "angVel", "cos_diff"])
for fish in tqdm(range(fish_num)):
    if fish in fish_exclude:
        continue
    exp = LotrExperiment(DATA_FOLDERS[fish])
    cell_num = len(exp.hdn_indexes)
    theta_rotation = get_bouts_props_array(exp.n_pts, exp.bouts_df, min_bias=TURN_BIAS, selection="all", value="bias")
    fictive_head = convolve_with_tau(np.cumsum(theta_rotation), tau * exp.fn, n_kernel_pts=20*exp.fn)
    velocity = convolve_with_tau(theta_rotation, tau * exp.fn, n_kernel_pts=20*exp.fn)
    trace = exp.noZ_traces[:, exp.hdn_indexes]
    
    # Calculate cos difference between the cell angle and the network phase
    cell_angle_2d = np.repeat(exp.rpc_angles.reshape(1,-1) , exp.n_pts, axis=0)
    network_phase_2d = exp.network_phase.reshape(-1,1).repeat(cell_num, axis=1)
    cos_diff = np.cos(cell_angle_2d - network_phase_2d)

    # generate the fish id and cell id for the df
    fish_tpr = np.array([fish] * (trace.shape[1] * trace.shape[0]))
    cell_tpr = np.repeat(np.arange(trace.shape[1]), trace.shape[0])
    vel_tpr = np.tile(velocity, trace.shape[1])
    trace_tpr = trace.T.flatten()
    cos_diff_tpr = cos_diff.T.flatten()
    df_tpr = pd.DataFrame({"fish": fish_tpr, "cell": cell_tpr, "activity": trace_tpr, "angVel": vel_tpr, 
                           "cos_diff": cos_diff_tpr})
    
    df_tpr.dropna(inplace=True, ignore_index=True)
    df = pd.concat([df,df_tpr], ignore_index=True)
    
df.to_pickle(FISH_RESULT_PATH / "ruben_data_timeseries_cos.pkl")

  0%|          | 0/31 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:38<00:00,  1.25s/it]


In [ ]:
df = pd.read_csv(FISH_RESULT_PATH / "ruben_data_timeseries_cos.csv")
df['angVel_abs'] = np.abs(df['angVel'])
df['angVel_square'] = np.square(df['angVel'])
df['rotation_direction'] = np.sign(df['angVel']).astype(int)
fish_num2 = len(df['fish'].unique())

In [13]:
df

,fish,cell,activity,angVel,cos_diff,angVel_abs,angVel_square,rotation_direction
0,0,0,0.594712,0.0,-0.624669,0.0,0.0,0
1,0,0,0.722613,0.0,-0.556828,0.0,0.0,0
2,0,0,0.730533,0.0,-0.573128,0.0,0.0,0
3,0,0,0.737218,0.0,-0.573651,0.0,0.0,0
4,0,0,0.741235,0.0,-0.570759,0.0,0.0,0
...,...,...,...,...,...,...,...,...
23395995,30,89,0.901598,0.0,0.583588,0.0,0.0,0
23395996,30,89,0.889405,0.0,0.578445,0.0,0.0,0
23395997,30,89,0.888340,0.0,0.602472,0.0,0.0,0
23395998,30,89,0.868808,0.0,0.586116,0.0,0.0,0


# Linear regression

In [ ]:
# Linear regression
models = np.zeros((fish_num), dtype=object)
results = np.zeros((fish_num), dtype=object)
for fish in tqdm(range(fish_num)):
    
    if fish in fish_exclude:
        continue

    df_fish = df.loc[df['fish'] == fish]
    cell_num = len(df_fish['cell'].unique())
    models_cell = np.zeros((cell_num), dtype=object)
    results_cell = np.zeros((cell_num), dtype=object)
    
    for celli in range(cell_num):
        df_cell = df_fish.loc[df_fish['cell'] == celli]
        if len(df_cell) > 0:
            model = smf.ols("activity ~ angVel_abs + angVel_abs : rotation_direction + cos_diff", data=df_cell)
            result = model.fit()
            
            models_cell[celli] = model
            results_cell[celli] = result
            # print(f'fly {fish+1}, cell {celli+1}:')
            # print(result.summary())
            
    models[fish] = models_cell
    results[fish] = results_cell

100%|██████████| 31/31 [02:19<00:00,  4.49s/it]


# Dataframe of statistical results and plots

In [ ]:
fish_col = np.array([])
cell_col = np.array([])

beta1_col = np.array([]) # angVel_abs coeff
beta1_p_col = np.array([]) 
beta1_u_95ci_col = np.array([])
beta1_l_95ci_col = np.array([])

beta2_col = np.array([]) # angVel_abs X rotation_direction coeff
beta2_p_col = np.array([])
beta2_u_95ci_col = np.array([])
beta2_l_95ci_col = np.array([])

offset_col = np.array([])

beta3_col = np.array([]) # cos_diff coeff
beta3_p_col = np.array([])
beta3_u_95ci_col = np.array([])
beta3_l_95ci_col = np.array([])

for fish in tqdm(range(fish_num)):

    if fish in fish_exclude:
        continue

    cell_num = len(df.loc[df['fish'] == fish]['cell'].unique())
    
    for celli in range(cell_num):

        if results[fish][celli] == 0:
            continue
        fish_col = np.append(fish_col, fish)
        cell_col = np.append(cell_col, celli)
        
        beta1_col = np.append(beta1_col, results[fish][celli].params['angVel_abs'])
        beta1_p_col = np.append(beta1_p_col, results[fish][celli].pvalues['angVel_abs'])
        beta1_u_95ci_col = np.append(beta1_u_95ci_col, results[fish][celli].conf_int().loc['angVel_abs'][1])
        beta1_l_95ci_col = np.append(beta1_l_95ci_col, results[fish][celli].conf_int().loc['angVel_abs'][0])
        
        beta2_col = np.append(beta2_col, results[fish][celli].params['angVel_abs:rotation_direction'])
        beta2_p_col = np.append(beta2_p_col, results[fish][celli].pvalues['angVel_abs:rotation_direction'])
        beta2_u_95ci_col = np.append(beta2_u_95ci_col, results[fish][celli].conf_int().loc['angVel_abs:rotation_direction'][1])
        beta2_l_95ci_col = np.append(beta2_l_95ci_col, results[fish][celli].conf_int().loc['angVel_abs:rotation_direction'][0])

        offset_col = np.append(offset_col, results[fish][celli].params['Intercept'])

        beta3_col = np.append(beta3_col, results[fish][celli].params['cos_diff'])
        beta3_p_col = np.append(beta3_p_col, results[fish][celli].pvalues['cos_diff'])
        beta3_u_95ci_col = np.append(beta3_u_95ci_col, results[fish][celli].conf_int().loc['cos_diff'][1])
        beta3_l_95ci_col = np.append(beta3_l_95ci_col, results[fish][celli].conf_int().loc['cos_diff'][0])

stat_df = pd.DataFrame({'fish': fish_col, 'cell': cell_col, 'offset': offset_col,  
                        'beta1': beta1_col, 'beta1_p': beta1_p_col, 'beta1_u_95ci': beta1_u_95ci_col, 'beta1_l_95ci': beta1_l_95ci_col, 
                        'beta2': beta2_col, 'beta2_p': beta2_p_col, 'beta2_u_95ci': beta2_u_95ci_col, 'beta2_l_95ci': beta2_l_95ci_col,
                        'beta3': beta3_col, 'beta3_p': beta3_p_col, 'beta3_u_95ci': beta3_u_95ci_col, 'beta3_l_95ci': beta3_l_95ci_col})

# correct for multiple comparisons
pvals = stat_df['beta1_p']
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method='Bonferroni')
stat_df['beta1_p_corrected'] = pvals_corrected

pvals = stat_df['beta2_p']
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method='Bonferroni')
stat_df['beta2_p_corrected'] = pvals_corrected

pvals = stat_df['beta3_p']
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method='Bonferroni')
stat_df['beta3_p_corrected'] = pvals_corrected

stat_df['ring'] = ['center'] * len(stat_df)
stat_df.loc[    (stat_df['beta2_p_corrected'] < 0.05) & (stat_df['beta2'] > 0)   , 'ring'] = 'right'
stat_df.loc[    (stat_df['beta2_p_corrected'] < 0.05) & (stat_df['beta2'] < 0)   , 'ring'] = 'left'

stat_df = add_coor_preferred_angle_to_lm_HD(stat_df)
stat_df = add_speed_modu_2_lm_df(stat_df)

stat_df['speed_modu'] = pd.Categorical(stat_df['speed_modu'], categories=['Increase', 'None', 'Decrease'], ordered=True)
stat_df['ring'] = pd.Categorical(stat_df['ring'], categories=['center', 'left', 'right'], ordered=True)

stat_df['anatomical_loc'] = np.where(stat_df.x > 0, 'right', 'left')
stat_df.to_pickle(FISH_RESULT_PATH / "ruben_lm_result_cos.pkl")

100%|██████████| 31/31 [00:33<00:00,  1.07s/it]
